# Setup and Load dataset



In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar 14 09:30:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:

!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.9 MB/s eta 0:00:00


In [4]:
#!pip install transformers

In [5]:
!pip install sentence_transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=44606e9c8a610bc5df92c054f7568b633071c7d1586fc72398afae6ad20483a1
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [6]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [7]:
!gdown --id 1YCgvm0zBUjCse9iPcwHpDzed9HfD5pna

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1YCgvm0zBUjCse9iPcwHpDzed9HfD5pna
To: /content/tmdb.zip
100% 9.32M/9.32M [00:00<00:00, 19.5MB/s]


In [8]:

!gdown --id 1aB6WIWCkra-MenkhRLVkR7elR_8GsbmJ

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1aB6WIWCkra-MenkhRLVkR7elR_8GsbmJ
To: /content/ml-latest-small.zip
100% 978k/978k [00:00<00:00, 63.2MB/s]


In [9]:
!unzip -o "tmdb.zip"  -d  "/content"

Archive:  tmdb.zip
  inflating: /content/tmdb_5000_credits.csv  
  inflating: /content/tmdb_5000_movies.csv  


In [10]:
!unzip -o "ml-latest-small.zip"  -d  "/content"

Archive:  ml-latest-small.zip
   creating: /content/ml-latest-small/
  inflating: /content/ml-latest-small/links.csv  
  inflating: /content/ml-latest-small/tags.csv  
  inflating: /content/ml-latest-small/ratings.csv  
  inflating: /content/ml-latest-small/README.txt  
  inflating: /content/ml-latest-small/movies.csv  


## Data Loading

In [11]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

#### Name of the file which contain all the item properties

In [12]:
content_df  = pd.read_csv( "ml-latest-small/movies.csv")

In [13]:
tmdb_df= pd.read_csv( "tmdb_5000_movies.csv")

In [14]:
# extract genres
import ast
def convert(obj):
    Lis=[]
    for i in ast.literal_eval(obj):
        Lis.append(i['name'])
    return Lis

In [15]:
tmdb_df['newgenres']=tmdb_df['genres'].apply(convert)

In [16]:
tmdb_df=tmdb_df[['id','newgenres','tagline','overview','original_title']]

In [17]:
link_df= pd.read_csv( "ml-latest-small/links.csv")

In [18]:
link_df=link_df.merge(tmdb_df,left_on='tmdbId',right_on='id')

In [19]:
content_df=content_df.merge(link_df,on="movieId")

In [20]:
content_df.columns

Index(['movieId', 'title', 'genres', 'imdbId', 'tmdbId', 'id', 'newgenres',
       'tagline', 'overview', 'original_title'],
      dtype='object')

In [21]:
content_df=content_df[['movieId', 'title',  'newgenres','tagline', 'overview']]

In [22]:
content_df.rename(columns = {'newgenres':'genere'}, inplace = True)

In [23]:
content_df['tagline'] = content_df['tagline'].fillna(' ')


In [24]:
rating_df= pd.read_csv( "ml-latest-small/ratings.csv")

In [25]:
rating_df.rename(columns = {'movieId':'itemID'}, inplace = True)
rating_df.rename(columns = {'userId':'userID'}, inplace = True)
content_df.rename(columns = {'movieId':'itemID'}, inplace = True)

In [26]:

# y = rating_df.groupby('itemID').count()['rating']>=250
# ids = y[y].index
# rating_df = rating_df[rating_df['itemID'].isin(ids)]

# y = rating_df.groupby('userID').count()['rating']>=200
# ids = y[y].index
# rating_df = rating_df[rating_df['userID'].isin(ids)]

In [27]:
u_items=list(rating_df.itemID.values)

In [28]:
content_df=content_df[content_df.itemID.isin(u_items)]

In [29]:
list_geners=[]
for i,row in content_df.iterrows():
  for gn in row['genere']:
      if gn not in list_geners:
        list_geners.append(gn)

In [30]:
list_geners

['Animation',
 'Comedy',
 'Family',
 'Adventure',
 'Action',
 'Thriller',
 'Drama',
 'Romance',
 'History',
 'Crime',
 'Fantasy',
 'War',
 'Mystery',
 'Music',
 'Horror',
 'Science Fiction',
 'Documentary',
 'Western',
 'Foreign',
 'TV Movie']

In [31]:
def listtostr(s):
  listToStr = ' '.join([str(elem) for elem in s])
  return listToStr

In [32]:
content_df['genere_str']=content_df['genere'].apply(listtostr)

<ipython-input-32-eca63d06b5e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['genere_str']=content_df['genere'].apply(listtostr)


In [33]:
content_df.columns

Index(['itemID', 'title', 'genere', 'tagline', 'overview', 'genere_str'], dtype='object')

## Details about dataset

In [34]:
itemid="itemID"

In [35]:
features=['title','genere_str', 'tagline', 'overview']

In [36]:
allcols=[itemid]
for i in features:
  allcols.append(i)
allcols.append('title')

In [37]:
len(content_df.itemID.values)

3536

# Setup

In [38]:
content_df['NewTag']=""
for i in features:
  content_df[i] = content_df[i].fillna(' ')
for i in features:
  content_df['NewTag']+=(" "+content_df[i])
content_df['NewTag']=content_df['NewTag'].astype(str)

In [39]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [40]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [41]:
def clean_text(text):
    
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub( re.compile("\'s"), ' ', text)
    text = re.sub(re.compile("\\r\\n"), ' ', text)
    text = re.sub(re.compile(r"[^\w\s]"), ' ', text)
    return text

In [42]:
stopwords=set(stopwords.words('english'))

In [43]:
def tokenizer(sentence, min_words=4, max_words=200, stopwords=stopwords, lemmatize=True):
    
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    

In [44]:
content_df['clean'] = content_df['NewTag'].apply(clean_text)
# content_df['token_lem_sentence'] = content_df['clean'].apply(
#         lambda x: tokenizer(x))

# Model train

In [45]:
from sentence_transformers import SentenceTransformer

tmodel = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')


In [46]:
import torch
#Select the device

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tmodel.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
)

In [47]:
corpus_embeddings = tmodel.encode(content_df.NewTag.values,show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/111 [00:00<?, ?it/s]

# Semantic search

In [48]:
from sentence_transformers import  util
TOP_K=10

In [49]:

query="pirates of the police officer"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=TOP_K)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
pd.set_option('display.max_colwidth', None)
content_df[content_df[itemid].isin(results)]['title']

400                                     Cop Land (1997)
515                            On the Waterfront (1954)
649                               Police Academy (1984)
702                               Corruptor, The (1999)
1192                                 Pootie Tang (2001)
2305    Pirates of the Caribbean: At World's End (2007)
3024               Pirates! Band of Misfits, The (2012)
3175                                    R.I.P.D. (2013)
3182                                      2 Guns (2013)
3400                               Run All Night (2015)
Name: title, dtype: object

In [50]:

query="a movie about robots"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=TOP_K)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
pd.set_option('display.max_colwidth', None)
content_df[content_df[itemid].isin(results)]['title']

843                              Creepshow (1982)
1012                            Titan A.E. (2000)
1083                          6th Day, The (2000)
1308            Bill & Ted's Bogus Journey (1991)
1602    Terminator 3: Rise of the Machines (2003)
1808                              I, Robot (2004)
2320                          Transformers (2007)
2430                       Superhero Movie (2008)
3060                         Robot & Frank (2012)
3244                        The Lego Movie (2014)
Name: title, dtype: object

In [51]:

query="a movie about Artificial Intelligence"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=TOP_K)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
pd.set_option('display.max_colwidth', None)
content_df[content_df[itemid].isin(results)]['title']

64                      Species (1995)
396                     Contact (1997)
1012                 Titan A.E. (2000)
1823    AVP: Alien vs. Predator (2004)
2462                  Meet Dave (2008)
2789                 Jackass 3D (2010)
3089      Indie Game: The Movie (2012)
3299                  I Origins (2014)
3302                Signal, The (2014)
3360              The Interview (2014)
Name: title, dtype: object

In [52]:

query="a movie about pirates"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=TOP_K)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
pd.set_option('display.max_colwidth', None)
content_df[content_df[itemid].isin(results)]['title']

1604    Pirates of the Caribbean: The Curse of the Black Pearl (2003)
1958                                                    Sahara (2005)
2036                                             Into the Blue (2005)
2305                  Pirates of the Caribbean: At World's End (2007)
2430                                           Superhero Movie (2008)
2868               Pirates of the Caribbean: On Stranger Tides (2011)
2915                                            Shark Night 3D (2011)
3024                             Pirates! Band of Misfits, The (2012)
3244                                            The Lego Movie (2014)
3346                                    Penguins of Madagascar (2014)
Name: title, dtype: object

# Search with FAISS

In [53]:
import faiss
from sentence_transformers import SentenceTransformer
encoded_data = corpus_embeddings
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(corpus_embeddings.shape[1]))
index.add_with_ids(encoded_data, np.array(range(0, len(content_df))))
faiss.write_index(index, 'item.index')

### fetch details of the document

In [54]:

def search(query, k, index, model):
    qv = model.encode([query])
    top_k = index.search(qv, k)
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [content_df.iloc[idx][itemid] for idx in top_k_ids]
    return results

In [55]:
from pprint import pprint
query="movie about action and drama"
results=search(query, k=10, index=index, model=tmodel)
content_df[content_df[itemid].isin(results)]['title']

130                                                     Last Action Hero (1993)
244                                                             Die Hard (1988)
1006                                                      Action Jackson (1988)
1142                                                          15 Minutes (2001)
2240                                                       Blood Diamond (2006)
2430                                                     Superhero Movie (2008)
2486                                                      Disaster Movie (2008)
2511                                                          RocknRolla (2008)
2561    Good, the Bad, the Weird, The (Joheunnom nabbeunnom isanghannom) (2008)
2827                                                  Warrior's Way, The (2010)
Name: title, dtype: object

In [56]:
from pprint import pprint
query="a movie about robots"
results=search(query, k=10, index=index, model=tmodel)
content_df[content_df[itemid].isin(results)]['title']

843                              Creepshow (1982)
1012                            Titan A.E. (2000)
1308            Bill & Ted's Bogus Journey (1991)
1602    Terminator 3: Rise of the Machines (2003)
1808                              I, Robot (2004)
2263                            Epic Movie (2007)
2320                          Transformers (2007)
3060                         Robot & Frank (2012)
3244                        The Lego Movie (2014)
3330                            Big Hero 6 (2014)
Name: title, dtype: object

In [57]:
from pprint import pprint
query="a movie about pirates"
results=search(query, k=10, index=index, model=tmodel)
content_df[content_df[itemid].isin(results)]['title']

837                                     Thunderball (1965)
1958                                         Sahara (2005)
2036                                  Into the Blue (2005)
2305       Pirates of the Caribbean: At World's End (2007)
2370                                Southland Tales (2006)
2868    Pirates of the Caribbean: On Stranger Tides (2011)
3024                  Pirates! Band of Misfits, The (2012)
3205                               Captain Phillips (2013)
3244                                 The Lego Movie (2014)
3346                         Penguins of Madagascar (2014)
Name: title, dtype: object

In [58]:
from pprint import pprint
query="a movie about Artificial Intelligence"
results=search(query, k=10, index=index, model=tmodel)
content_df[content_df[itemid].isin(results)]['title']

64                    Species (1995)
396                   Contact (1997)
425                   Gattaca (1997)
2134              Renaissance (2006)
2263               Epic Movie (2007)
2462                Meet Dave (2008)
2789               Jackass 3D (2010)
3089    Indie Game: The Movie (2012)
3302              Signal, The (2014)
3360            The Interview (2014)
Name: title, dtype: object

# Recommendation based ranking

In [60]:
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.0/339.0 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 KB 35.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 52.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 KB 21.2 MB/s eta 0:00:00
     ━━━━━

In [61]:
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.sar import SAR
import cornac
from recommenders.datasets.python_splitters import python_random_split,python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED


##BPR

In [62]:
df=rating_df.copy()
df=df[['userID','itemID','rating']]
train=df
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)
bpr = cornac.models.BPR(
    k=200,
    max_iter=100,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=42
)

bpr.fit(train_set)

all_predictions = predict_ranking(bpr, train, usercol='userID', itemcol='itemID', remove_seen=False)



  0%|          | 0/100 [00:00<?, ?it/s]

Optimization finished!


In [63]:

k = len(u_items)
top_k_items = (
            all_predictions.sort_values(['userID', 'prediction'], ascending=[True, False])
            .groupby('userID', as_index=False)
            .head(k)
            .reset_index(drop=True)
        )

In [64]:
top_k_items=top_k_items.merge(content_df[['itemID','title']],on="itemID")

In [65]:
top_k_items=top_k_items.sort_values(['userID', 'prediction'], ascending=[True, False])

## SEARCH FOR USER #1

In [66]:
userquery=1

top 10 recommendations 

In [67]:
top_k_reco=top_k_items[top_k_items.userID==userquery]


### search1 

In [68]:
from sentence_transformers import SentenceTransformer, util
import torch


In [69]:
torch.device('cpu')

device(type='cpu')

In [70]:
query="a movie about pirates"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
search_list =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
top_k_reco[top_k_reco.itemID.isin(search_list)].sort_values(by='prediction',ascending=[False])['title'].head(10)

31110                                          Starship Troopers (1997)
65880                                                       Jaws (1975)
148840                                       Conan the Barbarian (1982)
150670                                          Last Action Hero (1993)
211670                                               Thunderball (1965)
271450                                                    Jaws 2 (1978)
301950    Pirates of the Caribbean: The Curse of the Black Pearl (2003)
557540                                            Street Fighter (1994)
613050                Pirates of the Caribbean: Dead Man's Chest (2006)
759450                  Pirates of the Caribbean: At World's End (2007)
Name: title, dtype: object

In [71]:
query="a movie about police officer"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
search_list =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
top_k_reco[top_k_reco.itemID.isin(search_list)].sort_values(by='prediction',ascending=[False])['title'].head(10)

9760                   Die Hard (1988)
78690     Beverly Hills Cop III (1994)
143960               Dick Tracy (1990)
250100                 Cop Land (1997)
315370     Beverly Hills Cop II (1987)
373320           Police Academy (1984)
666120                 One, The (2001)
753350               15 Minutes (2001)
838140           Running Scared (2006)
917440                 Joy Ride (2001)
Name: title, dtype: object

In [72]:
query="a movie about robots"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
search_list =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
top_k_reco[top_k_reco.itemID.isin(search_list)].sort_values(by='prediction',ascending=[False])['title'].head(10)

3660                             Matrix, The (1999)
11590                                  Alien (1979)
239120                            Titan A.E. (2000)
330010            Bill & Ted's Bogus Journey (1991)
336110                            Flatliners (1990)
385520                             Creepshow (1982)
407480                          6th Day, The (2000)
442860                              eXistenZ (1999)
484950                              I, Robot (2004)
510570    Terminator 3: Rise of the Machines (2003)
Name: title, dtype: object

In [73]:
query="a movie about Artificial Intelligence"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=30)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
search_list =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
top_k_reco[top_k_reco.itemID.isin(search_list)].sort_values(by='prediction',ascending=[False])['title'].head(10)

70150                    Alien: Resurrection (1997)
92110                                Gattaca (1997)
114680                               Contact (1997)
157380                               Species (1995)
225090                               Hackers (1995)
239120                            Titan A.E. (2000)
285480                       Man on the Moon (1999)
407480                          6th Day, The (2000)
442860                              eXistenZ (1999)
510570    Terminator 3: Rise of the Machines (2003)
Name: title, dtype: object

## SEARCH FOR USER #2

In [74]:
userquery=2

top 10 recommendations 

In [75]:
top_k_reco=top_k_items[top_k_items.userID==userquery]


### search1 

In [76]:
from sentence_transformers import SentenceTransformer, util
import torch


In [77]:
torch.device('cpu')

device(type='cpu')

In [78]:
query="a movie about pirates"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
search_list =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
top_k_reco[top_k_reco.itemID.isin(search_list)].sort_values(by='prediction',ascending=[False])['title'].head(10)

301951     Pirates of the Caribbean: The Curse of the Black Pearl (2003)
613051                 Pirates of the Caribbean: Dead Man's Chest (2006)
759451                   Pirates of the Caribbean: At World's End (2007)
65881                                                        Jaws (1975)
1839151                                            The Lego Movie (2014)
31111                                           Starship Troopers (1997)
1774491               Pirates of the Caribbean: On Stranger Tides (2011)
1487791                                          Captain Phillips (2013)
150671                                           Last Action Hero (1993)
148841                                        Conan the Barbarian (1982)
Name: title, dtype: object

In [79]:
query="a movie about police officer"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
search_list =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
top_k_reco[top_k_reco.itemID.isin(search_list)].sort_values(by='prediction',ascending=[False])['title'].head(10)

9761                    Die Hard (1988)
1448751                    Drive (2011)
1605521           21 Jump Street (2012)
78691      Beverly Hills Cop III (1994)
1269411                  Super 8 (2011)
373321            Police Academy (1984)
1828781                  Sicario (2015)
1933701             End of Watch (2012)
666121                  One, The (2001)
2144761            Let's Be Cops (2014)
Name: title, dtype: object

In [80]:
query="a movie about robots"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
search_list =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
top_k_reco[top_k_reco.itemID.isin(search_list)].sort_values(by='prediction',ascending=[False])['title'].head(10)

3661                              Matrix, The (1999)
11591                                   Alien (1979)
1275511                            Big Hero 6 (2014)
484951                               I, Robot (2004)
1839151                        The Lego Movie (2014)
682591                           Transformers (2007)
510571     Terminator 3: Rise of the Machines (2003)
851561                                 Robots (2005)
1629311                                     9 (2009)
1652491                            Surrogates (2009)
Name: title, dtype: object

In [81]:
query="a movie about Artificial Intelligence"
query_embedding = tmodel.encode(query, convert_to_tensor=True)

correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=30)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
search_list =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
top_k_reco[top_k_reco.itemID.isin(search_list)].sort_values(by='prediction',ascending=[False])['title'].head(10)

92111                                 Gattaca (1997)
114681                                Contact (1997)
682591                           Transformers (2007)
510571     Terminator 3: Rise of the Machines (2003)
285481                        Man on the Moon (1999)
225091                                Hackers (1995)
157381                                Species (1995)
1629311                                     9 (2009)
2135611                         The Interview (2014)
70151                     Alien: Resurrection (1997)
Name: title, dtype: object

# Use recommendion list to give weitage to different genere

In [82]:

k = 20
top_20_items = (
            all_predictions.sort_values(['userID', 'prediction'], ascending=[True, False])
            .groupby('userID', as_index=False)
            .head(k)
            .reset_index(drop=True)
        )

In [83]:
top_20_items=top_20_items.merge(content_df[['itemID','title','genere']],on="itemID")

In [84]:
top_20_items=top_20_items.sort_values(['userID', 'prediction'], ascending=[True, False])

In [85]:
list_geners=[]
for i,row in content_df.iterrows():
  for gn in row['genere']:
      if gn not in list_geners:
        list_geners.append(gn)

In [86]:
list_geners

['Animation',
 'Comedy',
 'Family',
 'Adventure',
 'Action',
 'Thriller',
 'Drama',
 'Romance',
 'History',
 'Crime',
 'Fantasy',
 'War',
 'Mystery',
 'Music',
 'Horror',
 'Science Fiction',
 'Documentary',
 'Western',
 'Foreign',
 'TV Movie']

In [87]:
user_genere_map=dict()
u_users=list(rating_df.userID.unique())
for i in u_users:
   user_genere_map[i]=dict()
   for j in list_geners:
    user_genere_map[i][j]=0
for i,row in top_20_items.iterrows():
  uid=row['userID']
  for gn in row['genere']:
      user_genere_map[uid][gn]+=1

In [88]:
genere_list=dict()
for i in list_geners:
  genere_list[i]=[]

In [89]:
user_watch_hist = pd.DataFrame()

for uid in u_users:

  for i in list_geners:
    genere_list[i].append(user_genere_map[uid][i])

In [90]:
for i in list_geners:
  user_watch_hist[i] = genere_list[i]


In [91]:
user_watch_hist['userID']=list(u_users)

In [92]:
# import scipy.stats as stats
# genere_score=dict()
# for i in list_geners:
#   user_watch_hist[i]=stats.zscore(user_watch_hist[i])



In [93]:
def weight(i):
    alpha = 0.5
    return alpha*pow((1-alpha),10-i)
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

## search for user #1

In [94]:
userquery=1

In [95]:
dataforuser=user_watch_hist[user_watch_hist.userID==userquery]
x = np.asarray([dataforuser])
genre_weights = softmax(x)

In [96]:
genre_weights=genre_weights[0][0]

In [97]:
all_generes_encode=dict()
for i in list_geners:
  all_generes_encode[i]=tmodel.encode(['action'])
  all_generes_encode[i]=np.asarray(all_generes_encode[i].astype('float32'))

In [98]:
user_genere_encode=[]
index=0
for i in list_geners:
  user_genere_encode.append(all_generes_encode[i]*genre_weights[index])
  index+=1
user_encoded_vector=user_genere_encode[0]
for i in range(1,len(user_genere_encode)):
  user_encoded_vector+=i

In [99]:

user_encoded_vector = np.asarray(user_encoded_vector)
user_encoded_vector = user_encoded_vector.reshape((1,corpus_embeddings.shape[1]))

In [100]:
query="a movie about pirates"
query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
alltags=list(content_df[content_df[itemid].isin(results)]['NewTag'])


#Compute embeddings
tembeddings = tmodel.encode(alltags)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(user_encoded_vector, tembeddings)

#Find the pairs with the highest cosine similarity scores
titles = list(content_df[content_df[itemid].isin(results)]['title'])
ranked_user_behaviour = [{'Title':x ,'Score': y} for x,y in zip(titles,cosine_scores.numpy()[0])]
ranked_user_behaviour = sorted(ranked_user_behaviour, key=lambda x: x['Score'], reverse=True)
ranked_user_behaviour[:10]


[{'Title': 'Penguins of Madagascar (2014)', 'Score': 0.00359996},
 {'Title': 'Sausage Party (2016)', 'Score': 0.0011603498},
 {'Title': 'Southland Tales (2006)', 'Score': -0.00024240022},
 {'Title': 'Bandidas (2006)', 'Score': -0.005480806},
 {'Title': 'Sharknado (2013)', 'Score': -0.009026544},
 {'Title': 'Jaws 2 (1978)', 'Score': -0.014675306},
 {'Title': 'Stan Helsing (2009)', 'Score': -0.017459176},
 {'Title': 'Pirates! Band of Misfits, The (2012)', 'Score': -0.02334557},
 {'Title': 'Shark Night 3D (2011)', 'Score': -0.025306711},
 {'Title': 'The Lego Movie (2014)', 'Score': -0.02567318}]

In [101]:
query="a movie about robots"
query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
alltags=list(content_df[content_df[itemid].isin(results)]['NewTag'])


#Compute embeddings
tembeddings = tmodel.encode(alltags)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(user_encoded_vector, tembeddings)

#Find the pairs with the highest cosine similarity scores
titles = list(content_df[content_df[itemid].isin(results)]['title'])
ranked_user_behaviour = [{'Title':x ,'Score': y} for x,y in zip(titles,cosine_scores.numpy()[0])]
ranked_user_behaviour = sorted(ranked_user_behaviour, key=lambda x: x['Score'], reverse=True)
ranked_user_behaviour[:10]


[{'Title': 'Monster House (2006)', 'Score': 0.0032982163},
 {'Title': 'Sausage Party (2016)', 'Score': 0.0011603378},
 {'Title': 'Big Hero 6 (2014)', 'Score': -0.005773406},
 {'Title': 'Meet Dave (2008)', 'Score': -0.019040419},
 {'Title': 'Escape From Tomorrow (2013)', 'Score': -0.021280624},
 {'Title': 'The Lego Movie (2014)', 'Score': -0.025673203},
 {'Title': 'Robot & Frank (2012)', 'Score': -0.027256284},
 {'Title': 'Jackass 3D (2010)', 'Score': -0.027925434},
 {'Title': 'Creepshow (1982)', 'Score': -0.028103612},
 {'Title': "Bill & Ted's Bogus Journey (1991)", 'Score': -0.02908747}]

In [102]:
query="a movie about Artificial Intelligence"
query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
alltags=list(content_df[content_df[itemid].isin(results)]['NewTag'])


#Compute embeddings
tembeddings = tmodel.encode(alltags)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(user_encoded_vector, tembeddings)

#Find the pairs with the highest cosine similarity scores
titles = list(content_df[content_df[itemid].isin(results)]['title'])
ranked_user_behaviour = [{'Title':x ,'Score': y} for x,y in zip(titles,cosine_scores.numpy()[0])]
ranked_user_behaviour = sorted(ranked_user_behaviour, key=lambda x: x['Score'], reverse=True)
ranked_user_behaviour[:10]


[{'Title': 'Epic Movie (2007)', 'Score': -0.010129329},
 {'Title': 'Meet Dave (2008)', 'Score': -0.019040436},
 {'Title': 'Escape From Tomorrow (2013)', 'Score': -0.02128066},
 {'Title': 'Signal, The (2014)', 'Score': -0.026184522},
 {'Title': 'Jackass 3D (2010)', 'Score': -0.027925435},
 {'Title': 'Superhero Movie (2008)', 'Score': -0.03403638},
 {'Title': 'Escape from Planet Earth (2013)', 'Score': -0.034764923},
 {'Title': 'The Interview (2014)', 'Score': -0.035186343},
 {'Title': '6th Day, The (2000)', 'Score': -0.036696866},
 {'Title': 'Indie Game: The Movie (2012)', 'Score': -0.03986738}]

In [103]:
query="a movie about police officer"
query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
alltags=list(content_df[content_df[itemid].isin(results)]['NewTag'])


#Compute embeddings
tembeddings = tmodel.encode(alltags)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(user_encoded_vector, tembeddings)

#Find the pairs with the highest cosine similarity scores
titles = list(content_df[content_df[itemid].isin(results)]['title'])
ranked_user_behaviour = [{'Title':x ,'Score': y} for x,y in zip(titles,cosine_scores.numpy()[0])]
ranked_user_behaviour = sorted(ranked_user_behaviour, key=lambda x: x['Score'], reverse=True)
ranked_user_behaviour[:10]


[{'Title': 'Joy Ride (2001)', 'Score': 0.0065333624},
 {'Title': 'Beverly Hills Cop III (1994)', 'Score': -0.0035072803},
 {'Title': '15 Minutes (2001)', 'Score': -0.010096139},
 {'Title': 'Police Academy: Mission to Moscow (1994)', 'Score': -0.010787674},
 {'Title': 'Adventures of Ford Fairlane, The (1990)', 'Score': -0.01615899},
 {'Title': 'Die Hard (1988)', 'Score': -0.017927352},
 {'Title': "Let's Be Cops (2014)", 'Score': -0.021435557},
 {'Title': 'Beverly Hills Cop II (1987)', 'Score': -0.021640826},
 {'Title': '16 Blocks (2006)', 'Score': -0.028868439},
 {'Title': 'Killer Joe (2011)', 'Score': -0.02944188}]

## search for user #1

In [104]:
userquery=2

In [105]:
dataforuser=user_watch_hist[user_watch_hist.userID==userquery]
x = np.asarray([dataforuser])
genre_weights = softmax(x)

In [106]:
genre_weights=genre_weights[0][0]

In [107]:
all_generes_encode=dict()
for i in list_geners:
  all_generes_encode[i]=tmodel.encode(['action'])
  all_generes_encode[i]=np.asarray(all_generes_encode[i].astype('float32'))

In [108]:
user_genere_encode=[]
index=0
for i in list_geners:
  user_genere_encode.append(all_generes_encode[i]*genre_weights[index])
  index+=1
user_encoded_vector=user_genere_encode[0]
for i in range(1,len(user_genere_encode)):
  user_encoded_vector+=i

In [109]:

user_encoded_vector = np.asarray(user_encoded_vector)
user_encoded_vector = user_encoded_vector.reshape((1,corpus_embeddings.shape[1]))

In [110]:
query="a movie about pirates"
query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
alltags=list(content_df[content_df[itemid].isin(results)]['NewTag'])


#Compute embeddings
tembeddings = tmodel.encode(alltags)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(user_encoded_vector, tembeddings)

#Find the pairs with the highest cosine similarity scores
titles = list(content_df[content_df[itemid].isin(results)]['title'])
ranked_user_behaviour = [{'Title':x ,'Score': y} for x,y in zip(titles,cosine_scores.numpy()[0])]
ranked_user_behaviour = sorted(ranked_user_behaviour, key=lambda x: x['Score'], reverse=True)
ranked_user_behaviour[:10]


[{'Title': 'Penguins of Madagascar (2014)', 'Score': 0.0036002053},
 {'Title': 'Sausage Party (2016)', 'Score': 0.0011606243},
 {'Title': 'Southland Tales (2006)', 'Score': -0.00024211616},
 {'Title': 'Bandidas (2006)', 'Score': -0.005480565},
 {'Title': 'Sharknado (2013)', 'Score': -0.00902627},
 {'Title': 'Jaws 2 (1978)', 'Score': -0.014674982},
 {'Title': 'Stan Helsing (2009)', 'Score': -0.017458957},
 {'Title': 'Pirates! Band of Misfits, The (2012)', 'Score': -0.023345299},
 {'Title': 'Shark Night 3D (2011)', 'Score': -0.02530646},
 {'Title': 'The Lego Movie (2014)', 'Score': -0.02567292}]

In [111]:
query="a movie about robots"
query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
alltags=list(content_df[content_df[itemid].isin(results)]['NewTag'])


#Compute embeddings
tembeddings = tmodel.encode(alltags)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(user_encoded_vector, tembeddings)

#Find the pairs with the highest cosine similarity scores
titles = list(content_df[content_df[itemid].isin(results)]['title'])
ranked_user_behaviour = [{'Title':x ,'Score': y} for x,y in zip(titles,cosine_scores.numpy()[0])]
ranked_user_behaviour = sorted(ranked_user_behaviour, key=lambda x: x['Score'], reverse=True)
ranked_user_behaviour[:10]


[{'Title': 'Monster House (2006)', 'Score': 0.0032984323},
 {'Title': 'Sausage Party (2016)', 'Score': 0.0011606175},
 {'Title': 'Big Hero 6 (2014)', 'Score': -0.0057731112},
 {'Title': 'Meet Dave (2008)', 'Score': -0.019040188},
 {'Title': 'Escape From Tomorrow (2013)', 'Score': -0.021280322},
 {'Title': 'The Lego Movie (2014)', 'Score': -0.025672931},
 {'Title': 'Robot & Frank (2012)', 'Score': -0.027256083},
 {'Title': 'Jackass 3D (2010)', 'Score': -0.027925149},
 {'Title': 'Creepshow (1982)', 'Score': -0.028103355},
 {'Title': "Bill & Ted's Bogus Journey (1991)", 'Score': -0.029087216}]

In [112]:
query="a movie about Artificial Intelligence"
query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
alltags=list(content_df[content_df[itemid].isin(results)]['NewTag'])


#Compute embeddings
tembeddings = tmodel.encode(alltags)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(user_encoded_vector, tembeddings)

#Find the pairs with the highest cosine similarity scores
titles = list(content_df[content_df[itemid].isin(results)]['title'])
ranked_user_behaviour = [{'Title':x ,'Score': y} for x,y in zip(titles,cosine_scores.numpy()[0])]
ranked_user_behaviour = sorted(ranked_user_behaviour, key=lambda x: x['Score'], reverse=True)
ranked_user_behaviour[:10]


[{'Title': 'Epic Movie (2007)', 'Score': -0.010129076},
 {'Title': 'Meet Dave (2008)', 'Score': -0.019040221},
 {'Title': 'Escape From Tomorrow (2013)', 'Score': -0.021280352},
 {'Title': 'Signal, The (2014)', 'Score': -0.026184201},
 {'Title': 'Jackass 3D (2010)', 'Score': -0.02792517},
 {'Title': 'Superhero Movie (2008)', 'Score': -0.034036074},
 {'Title': 'Escape from Planet Earth (2013)', 'Score': -0.03476462},
 {'Title': 'The Interview (2014)', 'Score': -0.035186123},
 {'Title': '6th Day, The (2000)', 'Score': -0.03669651},
 {'Title': 'Indie Game: The Movie (2012)', 'Score': -0.03986711}]

In [113]:
query="a movie about police officer"
query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
alltags=list(content_df[content_df[itemid].isin(results)]['NewTag'])


#Compute embeddings
tembeddings = tmodel.encode(alltags)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(user_encoded_vector, tembeddings)

#Find the pairs with the highest cosine similarity scores
titles = list(content_df[content_df[itemid].isin(results)]['title'])
ranked_user_behaviour = [{'Title':x ,'Score': y} for x,y in zip(titles,cosine_scores.numpy()[0])]
ranked_user_behaviour = sorted(ranked_user_behaviour, key=lambda x: x['Score'], reverse=True)
ranked_user_behaviour[:10]


[{'Title': 'Joy Ride (2001)', 'Score': 0.0065336726},
 {'Title': 'Beverly Hills Cop III (1994)', 'Score': -0.003507034},
 {'Title': '15 Minutes (2001)', 'Score': -0.010095813},
 {'Title': 'Police Academy: Mission to Moscow (1994)', 'Score': -0.010787387},
 {'Title': 'Adventures of Ford Fairlane, The (1990)', 'Score': -0.016158711},
 {'Title': 'Die Hard (1988)', 'Score': -0.017927008},
 {'Title': "Let's Be Cops (2014)", 'Score': -0.021435296},
 {'Title': 'Beverly Hills Cop II (1987)', 'Score': -0.021640558},
 {'Title': '16 Blocks (2006)', 'Score': -0.02886815},
 {'Title': 'Killer Joe (2011)', 'Score': -0.029441614}]

#CrossEncoder base Reranking

In [114]:
## Load our cross-encoder. Use fast tokenizer to speed up the tokenization
from sentence_transformers import CrossEncoder
cross_model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6', max_length=512)

In [115]:
def cross_score(model_inputs):
    scores = cross_model.predict(model_inputs)
    return scores


In [116]:
query="a movie about pirates"

query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
newtags=content_df[content_df[itemid].isin(results)]['NewTag']


model_inputs = [[query,item] for item in newtags]
scores = cross_score(model_inputs)
#Sort the scores in decreasing order
ranked_results = [{'Title': str(list(content_df[content_df[itemid]==inp]['title'])[0]), 'Score': score} for inp, score in zip(results, scores)]
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)
ranked_results

[{'Title': 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
  'Score': 0.7931788},
 {'Title': 'Sharknado (2013)', 'Score': 0.7867592},
 {'Title': 'Sausage Party (2016)', 'Score': 0.7534556},
 {'Title': 'Shark Night 3D (2011)', 'Score': 0.6190045},
 {'Title': 'The Lego Movie (2014)', 'Score': 0.23856884},
 {'Title': 'Starship Troopers (1997)', 'Score': 0.019074917},
 {'Title': 'Into the Blue (2005)', 'Score': 0.0028064824},
 {'Title': 'Southland Tales (2006)', 'Score': 0.00081469846},
 {'Title': 'Bandidas (2006)', 'Score': 0.00059217215},
 {'Title': 'Pirates! Band of Misfits, The (2012)', 'Score': 0.0005060551},
 {'Title': 'Thunderball (1965)', 'Score': 0.00036130974},
 {'Title': 'Captain Phillips (2013)', 'Score': 0.0003107272},
 {'Title': 'Penguins of Madagascar (2014)', 'Score': 0.00023878657},
 {'Title': 'Pirates of the Caribbean: On Stranger Tides (2011)',
  'Score': 0.00018886985},
 {'Title': 'Superhero Movie (2008)', 'Score': 0.00018759572},
 {'Title': 'Last Actio

In [117]:
query="a movie about Artificial Intelligence"

query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
newtags=content_df[content_df[itemid].isin(results)]['NewTag']


model_inputs = [[query,item] for item in newtags]
scores = cross_score(model_inputs)
#Sort the scores in decreasing order
ranked_results = [{'Title': str(list(content_df[content_df[itemid]==inp]['title'])[0]), 'Score': score} for inp, score in zip(results, scores)]
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)
ranked_results

[{'Title': 'Indie Game: The Movie (2012)', 'Score': 0.02130286},
 {'Title': 'Project Almanac (2015)', 'Score': 0.01649516},
 {'Title': 'Contact (1997)', 'Score': 0.0114077395},
 {'Title': 'Hackers (1995)', 'Score': 0.0015263422},
 {'Title': 'Escape from Planet Earth (2013)', 'Score': 0.0015190454},
 {'Title': 'Stealth (2005)', 'Score': 0.00091078744},
 {'Title': 'AVP: Alien vs. Predator (2004)', 'Score': 0.0006547431},
 {'Title': 'Meet Dave (2008)', 'Score': 0.00049323624},
 {'Title': 'Superhero Movie (2008)', 'Score': 0.00031385603},
 {'Title': 'Renaissance (2006)', 'Score': 0.00029239804},
 {'Title': 'Titan A.E. (2000)', 'Score': 0.0002894135},
 {'Title': 'Signal, The (2014)', 'Score': 0.0002614644},
 {'Title': 'Escape From Tomorrow (2013)', 'Score': 0.00023595622},
 {'Title': 'Bats (1999)', 'Score': 0.00023503967},
 {'Title': 'Gattaca (1997)', 'Score': 0.00023276873},
 {'Title': '6th Day, The (2000)', 'Score': 0.00018552746},
 {'Title': 'Jackass 3D (2010)', 'Score': 0.00018543056},


In [118]:
query="movie of the police officer"

query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
newtags=content_df[content_df[itemid].isin(results)]['NewTag']


model_inputs = [[query,item] for item in newtags]
scores = cross_score(model_inputs)
#Sort the scores in decreasing order
ranked_results = [{'Title': str(list(content_df[content_df[itemid]==inp]['title'])[0]), 'Score': score} for inp, score in zip(results, scores)]
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)
ranked_results

[{'Title': 'End of Watch (2012)', 'Score': 0.37222722},
 {'Title': 'Action Jackson (1988)', 'Score': 0.06464396},
 {'Title': 'Die Hard (1988)', 'Score': 0.06343874},
 {'Title': 'Dark Blue (2003)', 'Score': 0.010062984},
 {'Title': 'D.E.B.S. (2004)', 'Score': 0.0047165733},
 {'Title': 'Beverly Hills Cop II (1987)', 'Score': 0.004515525},
 {'Title': 'Super 8 (2011)', 'Score': 0.0018285539},
 {'Title': 'Takers (2010)', 'Score': 0.0017439249},
 {'Title': 'Righteous Kill (2008)', 'Score': 0.0017063487},
 {'Title': 'Sicario (2015)', 'Score': 0.0010602757},
 {'Title': 'Closer to the Moon (2013)', 'Score': 0.0010204031},
 {'Title': 'Adventures of Ford Fairlane, The (1990)', 'Score': 0.0010034542},
 {'Title': 'Police Academy: Mission to Moscow (1994)', 'Score': 0.001000614},
 {'Title': 'One, The (2001)', 'Score': 0.00086122885},
 {'Title': '2 Guns (2013)', 'Score': 0.00081017276},
 {'Title': 'Cop Land (1997)', 'Score': 0.00073816045},
 {'Title': '[REC]² (2009)', 'Score': 0.0005421087},
 {'Title

In [119]:
query="movie of the robots"

query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]
newtags=content_df[content_df[itemid].isin(results)]['NewTag']


model_inputs = [[query,item] for item in newtags]
scores = cross_score(model_inputs)
#Sort the scores in decreasing order
ranked_results = [{'Title': str(list(content_df[content_df[itemid]==inp]['title'])[0]), 'Score': score} for inp, score in zip(results, scores)]
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)
ranked_results

[{'Title': 'The Lego Movie (2014)', 'Score': 0.41753048},
 {'Title': 'Dawn of the Planet of the Apes (2014)', 'Score': 0.06545292},
 {'Title': 'Epic Movie (2007)', 'Score': 0.041046213},
 {'Title': 'Alien (1979)', 'Score': 0.028830633},
 {'Title': 'Sky Captain and the World of Tomorrow (2004)',
  'Score': 0.016315328},
 {'Title': 'Rise of the Planet of the Apes (2011)', 'Score': 0.012642347},
 {'Title': 'Terminator 3: Rise of the Machines (2003)', 'Score': 0.007638378},
 {'Title': 'Robots (2005)', 'Score': 0.0036070326},
 {'Title': 'Transformers: Revenge of the Fallen (2009)',
  'Score': 0.0033923825},
 {'Title': 'Titan A.E. (2000)', 'Score': 0.0033907562},
 {'Title': "Bill & Ted's Bogus Journey (1991)", 'Score': 0.00088590366},
 {'Title': 'Superhero Movie (2008)', 'Score': 0.0008275909},
 {'Title': 'Robot & Frank (2012)', 'Score': 0.0008228127},
 {'Title': 'Transformers: Dark of the Moon (2011)', 'Score': 0.00074686337},
 {'Title': 'Godzilla 2000 (Gojira ni-sen mireniamu) (1999)',
  '

# LAMDAMART

In [120]:
list_geners=[]
for i,row in content_df.iterrows():
  for gn in row['genere']:
      if gn not in list_geners:
        list_geners.append(gn)

In [121]:
list_geners

['Animation',
 'Comedy',
 'Family',
 'Adventure',
 'Action',
 'Thriller',
 'Drama',
 'Romance',
 'History',
 'Crime',
 'Fantasy',
 'War',
 'Mystery',
 'Music',
 'Horror',
 'Science Fiction',
 'Documentary',
 'Western',
 'Foreign',
 'TV Movie']

In [122]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

def create_genre_flags(df, popular_genres):
    df = df.dropna(subset=['genere'])
    df['genere'] = df['genere'].apply(lambda x:",".join(s.strip() for s in x))
    listofitems=dict()


    genre_df =df

    for i in popular_genres:
      genre_df[i]=[0] * df.shape[0]

    for i,row in genre_df.iterrows():
      for gn in str(row['genere']).split(','):
        row[gn]=1

        
    genre_df=genre_df.drop(['genere'], axis=1)
    return genre_df

In [123]:
content_df.head(1)

,itemID,title,genere,tagline,overview,genere_str,NewTag,clean
0,1,Toy Story (1995),"[Animation, Comedy, Family]",,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",Animation Comedy Family,"Toy Story (1995) Animation Comedy Family Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",toy story 1995 animation comedy family led by woody andy toys live happily in his room until andy birthday brings buzz lightyear onto the scene afraid of losing his place in andy heart woody plots against buzz but when circumstances separate buzz and woody from their owner the duo eventually learns to put aside their differences


In [124]:
genre_info_df = create_genre_flags(content_df,list_geners)

In [125]:
genre_info_df  = genre_info_df.dropna(axis=1)

In [126]:
genre_info_df.head(1)

,itemID,title,tagline,overview,genere_str,NewTag,clean,Animation,Comedy,Family,...,Fantasy,War,Mystery,Music,Horror,Science Fiction,Documentary,Western,Foreign,TV Movie
0,1,Toy Story (1995),,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",Animation Comedy Family,"Toy Story (1995) Animation Comedy Family Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",toy story 1995 animation comedy family led by woody andy toys live happily in his room until andy birthday brings buzz lightyear onto the scene afraid of losing his place in andy heart woody plots against buzz but when circumstances separate buzz and woody from their owner the duo eventually learns to put aside their differences,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score,coverage_error

from recommenders.datasets.python_splitters import python_random_split,  python_stratified_split

In [174]:
train, test = python_stratified_split(rating_df, 0.75)

In [175]:
train=train.merge(genre_info_df,on="itemID")
test=test.merge(genre_info_df,on="itemID")

In [176]:
train.sort_values(by='userID',inplace=True)
test.sort_values(by='userID',inplace=True)

In [178]:
features=['userID','itemID', 'Animation', 'Comedy',
       'Family', 'Adventure', 'Action', 'Thriller', 'Drama', 'Romance',
       'History', 'Crime', 'Fantasy', 'War', 'Mystery', 'Music', 'Horror',
       'Science Fiction', 'Documentary', 'Western', 'Foreign', 'TV Movie']

In [179]:
target = 'rating'

In [180]:

xtrain,ytrain = train[features],train[target].apply(lambda x:int(x))
xtest,ytest = test[features],test[target].apply(lambda x:int(x))


Index(['userID', 'itemID', 'rating', 'timestamp', 'title', 'tagline',
       'overview', 'genere_str', 'NewTag', 'clean', 'Animation', 'Comedy',
       'Family', 'Adventure', 'Action', 'Thriller', 'Drama', 'Romance',
       'History', 'Crime', 'Fantasy', 'War', 'Mystery', 'Music', 'Horror',
       'Science Fiction', 'Documentary', 'Western', 'Foreign', 'TV Movie'],
      dtype='object')

In [181]:
get_group_size = lambda df: df.reset_index().groupby("userID")['userID'].count()

In [182]:
train_groups = get_group_size(xtrain)
test_groups = get_group_size(xtest)

In [183]:
sum(train_groups) , sum(test_groups)

(52741, 17453)

In [184]:
from sklearn import preprocessing,model_selection,metrics
from lightgbm import LGBMRanker

model = LGBMRanker(objective="lambdarank",n_estimators=20)
model.fit(xtrain,ytrain,group=train_groups,eval_set=[(xtest,ytest)],eval_group=[test_groups],eval_metric=['map'])

[1]	valid_0's ndcg@1: 0.64463	valid_0's map@1: 0.990164
[2]	valid_0's ndcg@1: 0.647282	valid_0's map@1: 0.986885
[3]	valid_0's ndcg@1: 0.645656	valid_0's map@1: 0.988525
[4]	valid_0's ndcg@1: 0.645431	valid_0's map@1: 0.988525
[5]	valid_0's ndcg@1: 0.646968	valid_0's map@1: 0.988525
[6]	valid_0's ndcg@1: 0.645656	valid_0's map@1: 0.988525
[7]	valid_0's ndcg@1: 0.653779	valid_0's map@1: 0.988525
[8]	valid_0's ndcg@1: 0.650231	valid_0's map@1: 0.988525
[9]	valid_0's ndcg@1: 0.646875	valid_0's map@1: 0.988525
[10]	valid_0's ndcg@1: 0.650937	valid_0's map@1: 0.988525
[11]	valid_0's ndcg@1: 0.644379	valid_0's map@1: 0.988525
[12]	valid_0's ndcg@1: 0.650112	valid_0's map@1: 0.988525
[13]	valid_0's ndcg@1: 0.642839	valid_0's map@1: 0.986885
[14]	valid_0's ndcg@1: 0.649082	valid_0's map@1: 0.988525
[15]	valid_0's ndcg@1: 0.648264	valid_0's map@1: 0.988525
[16]	valid_0's ndcg@1: 0.649082	valid_0's map@1: 0.988525
[17]	valid_0's ndcg@1: 0.650986	valid_0's map@1: 0.988525
[18]	valid_0's ndcg@1: 0

LGBMRanker(n_estimators=20, objective='lambdarank')

In [185]:
candidate_pool = train['itemID'].unique().tolist()

In [186]:
user_2_item_df = train[['userID','itemID']]

In [187]:
user_2_item_map = dict(zip(user_2_item_df.index,user_2_item_df['itemID']))

In [188]:
item_id_2_name=content_df.drop_duplicates(subset=["itemID","title"])[['itemID',"title"]]

In [189]:
item_id_2_name_map = dict(zip(item_id_2_name['itemID'],item_id_2_name['title']))
item_id_2_name_map[1]

'Toy Story (1995)'

In [190]:
xtrain.columns

Index(['userID', 'itemID', 'Animation', 'Comedy', 'Family', 'Adventure',
       'Action', 'Thriller', 'Drama', 'Romance', 'History', 'Crime', 'Fantasy',
       'War', 'Mystery', 'Music', 'Horror', 'Science Fiction', 'Documentary',
       'Western', 'Foreign', 'TV Movie'],
      dtype='object')

In [191]:
all_cols_feature=xtrain[['itemID', 'Animation', 'Comedy', 'Family',
       'Adventure', 'Action', 'Thriller', 'Drama', 'Romance', 'History',
       'Crime', 'Fantasy', 'War', 'Mystery', 'Music', 'Horror',
       'Science Fiction', 'Documentary', 'Western', 'Foreign', 'TV Movie']]

In [192]:
all_cols_feature.head(1)

,itemID,Animation,Comedy,Family,Adventure,Action,Thriller,Drama,Romance,History,...,Fantasy,War,Mystery,Music,Horror,Science Fiction,Documentary,Western,Foreign,TV Movie
0,3578,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [194]:
all_cols_feature=all_cols_feature.drop_duplicates()

## Prediction for user id 1

In [221]:
all_cols_feature['userID']=1
predictions = pd.DataFrame()
predictions['itemID'] = np.array( candidate_pool)
predictions['name'] = np.array([item_id_2_name_map.get(id_) for id_ in candidate_pool])
predictions['score'] = model.predict(all_cols_feature)
predictions = predictions.sort_values(by='score',ascending=False)

<ipython-input-221-64b0a372e5f3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_cols_feature['userID']=1


In [220]:
predictions

,itemID,name,score
0,3578,Gladiator (2000),0.050787
2266,8666,Catwoman (2004),0.050787
2242,7369,Scooby-Doo 2: Monsters Unleashed (2004),0.050787
2243,68135,17 Again (2009),0.050787
2244,33499,Monster-in-Law (2005),0.050787
...,...,...,...
1128,8951,Vera Drake (2004),0.050787
1129,43936,16 Blocks (2006),0.050787
1130,62374,Body of Lies (2008),0.050787
1131,55118,Eastern Promises (2007),0.050787


In [227]:
query="a movie about pirates"

query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]

res=predictions[predictions[itemid].isin(results)]
res.sort_values(by='score',ascending=False)
res[['name']].head(10)

,name
2475,"Marine, The (2006)"
2451,Into the Blue (2005)
2385,Superhero Movie (2008)
1798,Sahara (2005)
2094,Street Fighter (1994)
1906,"Pirates! Band of Misfits, The (2012)"
2957,Sharknado (2013)
3310,Bandidas (2006)
2630,Stan Helsing (2009)
2810,Southland Tales (2006)


In [228]:
query="a movie about Artificial Intelligence"

query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]

res=predictions[predictions[itemid].isin(results)]
res.sort_values(by='score',ascending=False)
res[['name']].head(10)

,name
2131,Stealth (2005)
2461,Epic Movie (2007)
2347,Escape from Planet Earth (2013)
2385,Superhero Movie (2008)
1859,eXistenZ (1999)
2033,"Signal, The (2014)"
1899,Jackass 3D (2010)
3046,Project Almanac (2015)
3293,Renaissance (2006)
3362,Escape From Tomorrow (2013)


In [229]:
query="movie of the police officer"

query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]

res=predictions[predictions[itemid].isin(results)]
res.sort_values(by='score',ascending=False)
res[['name']].head(10)

,name
2132,15 Minutes (2001)
2524,R.I.P.D. (2013)
2502,Super 8 (2011)
1820,Closer to the Moon (2013)
1887,Paul Blart: Mall Cop (2009)
1774,Beverly Hills Cop II (1987)
1749,"One, The (2001)"
2025,Cop Land (1997)
2081,Killer Joe (2011)
1903,D.E.B.S. (2004)


In [230]:
query="movie of the robots"

query_embedding = tmodel.encode(query, convert_to_tensor=True)
alltitles=[]
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=25)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
results =  [content_df.iloc[idx][itemid] for idx in correct_hits_ids]

res=predictions[predictions[itemid].isin(results)]
res.sort_values(by='score',ascending=False)
res[['name']].head(10)

,name
2196,Godzilla 2000 (Gojira ni-sen mireniamu) (1999)
2208,Bill & Ted's Bogus Journey (1991)
2461,Epic Movie (2007)
2385,Superhero Movie (2008)
1868,Monsters vs. Aliens (2009)
2100,Transformers: Revenge of the Fallen (2009)
2075,Dawn of the Planet of the Apes (2014)
3293,Renaissance (2006)
2711,Transformers: Age of Extinction (2014)
2597,Creepshow (1982)
